# The Jindosh Riddle #
A solution to the Jindosh riddle using Python and constraints

In [ ]:
"""Solution to the Jindosh Riddle"""
import constraint

problem = constraint.Problem()

# Variable values
WOMEN = ['lady_winslow', 'doctor_marcolla', 'countess_contee', 'madam_natsiou', 'baroness_finch']
# Variable names
ITEMS = ["snuff_tin", "diamond", "bird_pendant", "war_medal", "ring"]  # Goal variables
SEATS = ["far_left", "mid_left", "center", "mid_right", "far_right"]
COLORS = ['green', 'purple', 'white', 'blue', 'red']
DRINKS = ['wine', 'beer', 'whiskey', 'absinthe', 'rum']
TOWNS = ['karnaca', 'dabovka', 'dunwall', 'fraeport', 'baleton']

# Some variables can directly be set to their correct values, rest list range of allowed values
problem.addVariable(ITEMS[0], WOMEN)
problem.addVariable(ITEMS[1], WOMEN[2:3])
problem.addVariable(ITEMS[2], WOMEN)
problem.addVariable(ITEMS[3], WOMEN)
problem.addVariable(ITEMS[4], WOMEN)

problem.addVariable(SEATS[0], WOMEN[3:4])
problem.addVariable(SEATS[1], WOMEN)
problem.addVariable(SEATS[2], WOMEN)
problem.addVariable(SEATS[3], WOMEN)
problem.addVariable(SEATS[4], WOMEN)

problem.addVariable(COLORS[0], WOMEN[1:2])
problem.addVariable(COLORS[1], WOMEN)
problem.addVariable(COLORS[2], WOMEN)
problem.addVariable(COLORS[3], WOMEN)
problem.addVariable(COLORS[4], WOMEN)

problem.addVariable(DRINKS[0], WOMEN)
problem.addVariable(DRINKS[1], WOMEN)
problem.addVariable(DRINKS[2], WOMEN[4:])
problem.addVariable(DRINKS[3], WOMEN)
problem.addVariable(DRINKS[4], WOMEN)

problem.addVariable(TOWNS[0], WOMEN)
problem.addVariable(TOWNS[1], WOMEN)
problem.addVariable(TOWNS[2], WOMEN)
problem.addVariable(TOWNS[3], WOMEN)
problem.addVariable(TOWNS[4], WOMEN[0:1])

def two_variables_equal_constraint(var0, var1):
    """Constraint that the variables must be equal"""
    if var0 == var1:
        return True


def is_neighbor(person_a, person_b, row):
    """Return True if a and b are seated next to each other in row"""
    result = False
    if person_a in row and person_b in row:
        if abs(row.index(person_a) - row.index(person_b)) == 1:
            result = True
    return result


def is_left_of(person_a, person_b, row):
    """True if a is directly left of b (index 0 is leftmost)"""
    result = False
    if person_a in row and person_b in row:
        if row.index(person_a) - row.index(person_b) == -1:
            result = True
    return result


def color_and_seating_constraint(purple, white, blue, far_left, mid_left, center, mid_right, far_right):
    """Constraints on colors and seating (2 rules)"""
    row = [far_left, mid_left, center, mid_right, far_right]
    if purple == mid_left and is_left_of(blue, white, row):
        return True


def item_town_drink_and_seating_constraint(ring, dunwall, beer, far_left, mid_left, center, mid_right, far_right):
    """Check an item, a town, a drink and how associated women are seated """
    row = [far_left, mid_left, center, mid_right, far_right]
    if ring != dunwall and is_neighbor(dunwall, ring, row) and dunwall != beer and is_neighbor(dunwall, beer, row):
        return True


def item_next_to_town(snuff_tin, karnaca, far_left, mid_left, center, mid_right, far_right):
    """An item (snuff tin) is next to a town (karnaca)"""
    row = [far_left, mid_left, center, mid_right, far_right]
    if snuff_tin != karnaca and is_neighbor(snuff_tin, karnaca, row):
        return True


problem.addConstraint(two_variables_equal_constraint, [COLORS[3], DRINKS[0]])
problem.addConstraint(two_variables_equal_constraint, [COLORS[4], TOWNS[0]])
problem.addConstraint(two_variables_equal_constraint, [DRINKS[3], TOWNS[3]])
problem.addConstraint(two_variables_equal_constraint, [DRINKS[4], SEATS[2]])
problem.addConstraint(two_variables_equal_constraint, [ITEMS[3], TOWNS[1]])

problem.addConstraint(color_and_seating_constraint, [COLORS[1], COLORS[2], COLORS[3]] + SEATS)
problem.addConstraint(item_town_drink_and_seating_constraint, [ITEMS[4], TOWNS[2], DRINKS[1]] + SEATS)
problem.addConstraint(item_next_to_town, [ITEMS[0], TOWNS[0]] + SEATS)

# Per group of 5 variables, a specific woman can only occur once.
# E.g. the same woman can not be in both rum and wine variables at same time.
problem.addConstraint(constraint.AllDifferentConstraint(), ITEMS)
problem.addConstraint(constraint.AllDifferentConstraint(), SEATS)
problem.addConstraint(constraint.AllDifferentConstraint(), COLORS)
problem.addConstraint(constraint.AllDifferentConstraint(), DRINKS)
problem.addConstraint(constraint.AllDifferentConstraint(), TOWNS)

_solutions = problem.getSolutions()


def print_solutions(solutions):
    """Print solution for items"""
    p_a, p_b, p_c, p_d, p_e = ITEMS[0], ITEMS[1], ITEMS[2], ITEMS[3], ITEMS[4]
    format_header = "{}\t{}\t\t{}\t{}\t{}"
    format_string = "{}\t{}\t{}\t{}\t{}"
    print(format_header.format(p_a, p_b, p_c, p_d, p_e))
    print(format_header.format('-' * len(p_a), '-' * len(p_b), '-' * len(p_c), '-' * len(p_d), '-' * len(p_e)))
    for sol_dict in solutions:
        print(format_string.format(sol_dict[p_a], sol_dict[p_b], sol_dict[p_c], sol_dict[p_d], sol_dict[p_e]))


if len(_solutions) == 1:
    print(_solutions)
elif len(_solutions) < 100:
    print_solutions(_solutions)
print("- - -\n{} solutions".format(len(_solutions)))